In [224]:
# Setup

import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GlyphRenderer, LinearColorMapper
from bokeh.io import push_notebook
from numba import jit, njit

from ipywidgets import interact

from scipy import optimize

output_notebook()

Loading BokehJS ...

In [247]:
# Models

def PolyModel(x, a, b, c):
	return a + (b*x) + (c*x*x)

def LinModel(x, m, c):
	return (m*x)+c

In [248]:
# M & C Calculation

def M_from_ms(arr):
    dep = arr[:,0]
    indep = arr[:,2]
    
    paramsLin, params_covarianceLin = optimize.curve_fit(LinModel, indep, dep)
    return paramsLin
    
def C_from_cs(arr):
    dep = arr[:,1]
    indep = arr[:,2]
    
    paramsLin, params_covarianceLin = optimize.curve_fit(LinModel, indep, dep)
    
    return paramsLin
    

In [249]:
# m & c Identification for Particular Temp

def hum_for_temp(t, arr):
    new_arr = np.array([[0,0,0]])

    for item in arr:
        temper = item[6] - t
        if (abs(temper) < 1.5):
            #print(item)
            new_arr = np.append(new_arr,[[item[3],item[6],item[7]]],axis = 0)

    log_dat = new_arr

    vhum = log_dat[1:,0] #Vaisala Hum,
    stemp = log_dat[1:,1] #Vaisala Hum,
    sraw = log_dat[1:,2] #SGS RAW,

    paramsLin, params_covarianceLin = optimize.curve_fit(LinModel, vhum, sraw)

    #print(paramsLin)
    return np.append(paramsLin, t)

In [250]:
# Temp Compensated Humidity Calculation

def calc_hum(Freq, _M, _C, temp):
    M = _M[0]*temp + _M[1]
    C = _C[0]*temp + _C[1]

    Humidity = (Freq - C)/ M
    return Humidity

In [258]:
# Data Acquisition

logData2 = pd.read_csv("test2.csv")
logData2.head(5)

log_data2 = logData2.to_numpy()

ehum = log_data2[1:,1] #E+E Hum,
etemp = log_data2[1:,2] #E+E Temp,
vhum = log_data2[1:,3] #Vaisala Hum,
vtemp = log_data2[1:,4] #Vaisala Temp,
shum = log_data2[1:,5] #SGS Hum,
stemp = log_data2[1:,6] #SGS Temp,
sraw = log_data2[1:,7] #SGS RAW,

data_arr = log_data2[4000:11000]

In [259]:
# M and C Identification

arr = np.array([hum_for_temp(10, data_arr), 
                hum_for_temp(20, data_arr), 
                hum_for_temp(23, data_arr), 
                hum_for_temp(25, data_arr), 
                hum_for_temp(34, data_arr), 
                hum_for_temp(39, data_arr)
               ])

p = figure(title="At Temp " + str(temp), x_axis_label='Humidity', y_axis_label='Raw')

for item in arr:
    print(item)
    p.line(vhum, LinModel(vhum, item[0], item[1]), legend_label="Linear", line_width=2)

print()
print("M : m ", M_from_ms(arr)[0], "  c ", M_from_ms(arr)[1])
print("C : m ", C_from_cs(arr)[0], "  c ", C_from_cs(arr)[1])

_M = M_from_ms(arr)
_C = C_from_cs(arr)

[-1.37628644e+02  5.44081321e+05  1.00000000e+01]
[-1.54570886e+02  5.43862852e+05  1.50000000e+01]
[-1.60177712e+02  5.42905296e+05  2.00000000e+01]
[-1.82976913e+02  5.43217631e+05  2.50000000e+01]
[-1.77336561e+02  5.41630436e+05  3.50000000e+01]
[-1.83760406e+02  5.40944781e+05  4.00000000e+01]

M : m  -1.4148380868420918   c  -131.88326651180546
C : m  -104.39274446499988   c  545296.5440632651


In [266]:
# Full Scale Difference

arr = data_arr

new_arr = np.array([[0,0,0,0,0]])

for item in arr:
    new_arr = np.append(new_arr,[[item[3],
                                  item[6], 
                                  item[7], 
                                  calc_hum(item[7], _M, _C, item[6]),
                                  item[3] - calc_hum(item[7], _M, _C, item[6])
                                 ]], axis = 0 
                       )

log_dat = new_arr

vhum = log_dat[1:,0] 
stemp = log_dat[1:,1] 
sraw = log_dat[1:,2] 
scalc = log_dat[1:,3] 
diff = log_dat[1:,4] 

p = figure(title="Calculated Humidity vs Vaisala Humidity ", x_axis_label='Vais Hum', y_axis_label='SGS Hum')

p.line(vhum, scalc, legend_label="Linear", line_width=2)

show(p)

mean_diff = np.mean(diff)

p = figure(title="Diff bw Vais and SGS, Mean: " + str(mean_diff), x_axis_label='Vais Hum', y_axis_label='SGS Hum')

p.line(vhum, diff, legend_label="Linear", line_width=2)

show(p)


In [267]:
# Temp dependent Difference

new_arr = np.array([[0,0,0,0,0]])

temp = 40

arr = data_arr

for item in arr:
    temper = item[6] - temp
    if (abs(temper) < 1.5):
        #print(item)
        new_arr = np.append(new_arr,[[item[3],
                                  item[6], 
                                  item[7], 
                                  calc_hum(item[7], _M, _C, item[6]),
                                  item[3] - calc_hum(item[7], _M, _C, item[6]) 
                                 ]], axis = 0 
                       )
    
vhum = new_arr[1:,0] 
stemp = new_arr[1:,1] 
sraw = new_arr[1:,2] 
scalc = new_arr[1:,3] 
diff = new_arr[1:,4] 

p = figure(title="Calculated Humidity vs Vaisala Humidity ", x_axis_label='Vais Hum', y_axis_label='SGS Hum')

p.line(vhum, scalc, legend_label="Linear", line_width=2)

show(p)

mean_diff = np.mean(diff)

p = figure(title="Diff bw Vais and SGS, Mean: " + str(mean_diff), x_axis_label='Vais Hum', y_axis_label='SGS Hum')

p.line(vhum, diff, legend_label="Linear", line_width=2)

show(p)

In [268]:
# Hum & Temp Variation Visualisation

new_arr = np.array([[0,0,0,0,0,0]])

arr = data_arr

count = -1
for item in arr:
    count = count + 1
    new_arr = np.append(new_arr,[[item[3],
                                  item[6], 
                                  item[7], 
                                  calc_hum(item[7], _M, _C, item[6]),
                                  abs(item[3] - calc_hum(item[7], _M, _C, item[6])),
                                  count
                                 ]], axis = 0 
                       )
    
vhum = new_arr[1:,0] #Vaisala Hum,
stemp = new_arr[1:,1] #Vaisala Hum,
sraw = new_arr[1:,2] #SGS RAW,
scalc = new_arr[1:,3] #SGS RAW,
diff = new_arr[1:,4] #SGS RAW,
count = new_arr[1:,5] #SGS RAW,

p = figure(title="Hum Var", x_axis_label='Count', y_axis_label='Hum')
p.line(count, shum, legend_label="Data", line_width=2, color="red")
show(p)

p = figure(title="Temp Variation", x_axis_label='Count', y_axis_label='Temp')
p.line(count, stemp, legend_label="Data", line_width=2, color="red")
show(p)

In [256]:
new_arr = np.array([[0,0,0,0]])

arr = data_arr

count = -1
for item in arr:
    count = count + 1
    new_arr = np.append(new_arr,[[item[3],item[6],item[7], count]],axis = 0)
    
vhum = new_arr[1:,0] #Vaisala Hum,
stemp = new_arr[1:,1] #Vaisala Hum,
sraw = new_arr[1:,2] #SGS RAW,
count = new_arr[1:,3] #SGS RAW,

paramsLin, params_covarianceLin = optimize.curve_fit(LinModel, vhum, sraw)

print(paramsLin)

p = figure(title="At Temp " + str(temp), x_axis_label='Humidity', y_axis_label='Raw')

p.line(count, sraw, legend_label="Data", line_width=2, color="red")

show(p)

[-1.99874994e+02  5.43542254e+05]
